Read in text from https://www.gutenberg.org/files/158/158-h/158-h.htm

In [1]:
!pwd

/home/ubuntu/Notebooks


In [1]:
import numpy as np
from numpy import random
import os
import gensim
import pickle
from nltk.tokenize.treebank import TreebankWordTokenizer

In [8]:
def get_austen_text():
    austen_text = ""
    indir = "/home/ubuntu/Notebooks/austenbooks"
    for root, dirs, filenames in os.walk(indir):
        for filename in filenames:
            if ".txt" in filename:
                path = indir+"/"+filename
                f = open(path)
                austen_text +=f.read()
    return austen_text

def get_text():
    slate_text = ""
    indir = '/home/ubuntu/Notebooks/OANC-GrAF/data/written_1/journal/slate'
    directories = []
    for root, dirs, filenames in os.walk(indir):
        directories.append(dirs)
    
    
    directories = directories[0]
    for folder in directories:
        indir = '/home/ubuntu/Notebooks/OANC-GrAF/data/written_1/journal/slate/'+folder
        for root, dirs, filenames in os.walk(indir):
            for filename in filenames:
                if ".txt" in filename:
                    path = indir+"/"+filename
                    f = open(path)
                    slate_text +=f.read()
    
    
    letters_text = ""
    indir = "/home/ubuntu/Notebooks/OANC-GrAF/data/written_1/letters/icic"
    for root, dirs, filenames in os.walk(indir):
        for filename in filenames:
            if ".txt" in filename:
                path = indir+"/"+filename
                f = open(path)
                letters_text +=f.read()
                
                
    eggan_text = ""
    path = "/home/ubuntu/Notebooks/OANC-GrAF/data/written_1/fiction/eggan/TheStory.txt"
    f = open(path)
    eggan_text +=f.read()
    
    
    verbatim_text = ""
    indir = "/home/ubuntu/Notebooks/OANC-GrAF/data/written_1/journal/verbatim"
    for root, dirs, filenames in os.walk(indir):
        for filename in filenames:
            if ".txt" in filename:
                path = indir+"/"+filename
                f = open(path)
                verbatim_text +=f.read()
                
    
    indir = '/home/ubuntu/Notebooks/OANC-GrAF/data/written_2/non-fiction/OUP'
    directories = []
    for root, dirs, filenames in os.walk(indir):
        directories.append(dirs)
    
    directories = directories[0]
    non_fiction_text = ""
    for folder in directories:
        indir = "/home/ubuntu/Notebooks/OANC-GrAF/data/written_2/non-fiction/OUP/"+folder
        for root, dirs, filenames in os.walk(indir):
            for filename in filenames:
                if ".txt" in filename:
                    path = indir+"/"+filename
                    f = open(path)
                    non_fiction_text +=f.read()
                    
    
    indir = '/home/ubuntu/Notebooks/OANC-GrAF/data/written_2/technical'
    directories = []
    for root, dirs, filenames in os.walk(indir):
        directories.append(dirs)
    
    directories = directories[0]
    technical_text = ""
    for folder in directories:
        indir = "/home/ubuntu/Notebooks/OANC-GrAF/data/written_2/technical/"+folder
        for root, dirs, filenames in os.walk(indir):
            for filename in filenames:
                if ".txt" in filename:
                    path = indir+"/"+filename
                    f = open(path)
                    technical_text +=f.read()
                    
                    
    gutenbooks = "/home/ubuntu/Notebooks/gutenberg/blake-poems.txt"
    f=open(path)
    gutenbooks += f.read()
    indir = "/home/ubuntu/Notebooks/gutenberg"
    for root, dirs, filenames in os.walk(indir):
        for filename in filenames:
            if (".txt" in filename) & ("chesterton" not in filename) & ("shakespeare" not in filename):
                path = indir+"/"+filename
                f = open(path)
                gutenbooks +=f.read()
                
    

    text = slate_text + letters_text + gutenbooks + technical_text + non_fiction_text + verbatim_text + eggan_text
    return text

In [9]:
other_text = get_text()
austen_text = get_austen_text()

In [12]:
import re
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("-", " ")
    
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "”" in text: text = text.replace("”","")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    text = text.replace("-", " ")
    text = text.replace('"', '')
    text = text.lower()
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences = [s.replace("“", "") for s in sentences]
    return sentences

# Train Neural Net

In [17]:
def vectorize_text(target_text, other_text):
    target_sentence_list = split_into_sentences(target_text)
    other_sentence_list = split_into_sentences(other_text)
    print('num of target, other sentences', len(target_sentence_list), len(other_sentence_list))
    
    word_tokenizer = TreebankWordTokenizer().tokenize
    word_set = set()
    for sentence in target_sentence_list+other_sentence_list:
        word_set = word_set.union(set(word_tokenizer(sentence)))
    print('number of words in vocab', len(word_set))
    word_list = list(word_set)
    num_to_word = {j:word for j, word in enumerate(word_list)}
    word_to_num = {word:j for j, word in enumerate(word_list)}
    
    target_vectorized_sentences = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)]) \
                                            for sentence in target_sentence_list])
    
    other_vectorized_sentences = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)]) \
                                            for sentence in other_sentence_list])
    success = True
    return target_sentence_list, other_sentence_list, target_vectorized_sentences, other_vectorized_sentences, num_to_word, word_to_num, word_list

In [ ]:
target_sentence_list, other_sentence_list, target_vectorized_sentences, other_vectorized_sentences,\
    num_to_word, word_to_num, word_list = vectorize_text(austen_text, other_text)

num of target, other sentences 35012 538166


In [ ]:
# with open('num_to_word.pkl', 'wb') as picklefile:
#     pickle.dump(num_to_word, picklefile)

# with open('word_to_num.pkl', 'wb') as picklefile:
#     pickle.dump(word_to_num, picklefile)

# with open('word_list.pkl', 'wb') as picklefile:
#     pickle.dump(word_list, picklefile)

# with open('target_vectorized_sentences.pkl', 'wb') as picklefile:
#     pickle.dump(target_vectorized_sentences, picklefile)

# with open('other_vectorized_sentences.pkl', 'wb') as picklefile:
#     pickle.dump(other_vectorized_sentences, picklefile)
    
# with open('target_sentence_list.pkl', 'wb') as picklefile:
#     pickle.dump(target_sentence_list, picklefile)

# with open('other_sentence_list.pkl', 'wb') as picklefile:
#     pickle.dump(other_sentence_list, picklefile)

In [3]:
def import_from_pickle():
    with open("target_sentence_list.pkl", 'rb') as picklefile: 
        target_sentence_list = pickle.load(picklefile)
    with open("other_sentence_list.pkl", 'rb') as picklefile: 
        other_sentence_list = pickle.load(picklefile)
    with open("target_vectorized_sentences.pkl", 'rb') as picklefile: 
        target_vectorized_sentences = pickle.load(picklefile)
    with open("other_vectorized_sentences.pkl", 'rb') as picklefile: 
        other_vectorized_sentences = pickle.load(picklefile)
        
    with open("word_to_num.pkl", 'rb') as picklefile: 
        word_to_num = pickle.load(picklefile)
    with open("word_list.pkl", 'rb') as picklefile: 
        word_list = pickle.load(picklefile)
    with open("num_to_word.pkl", 'rb') as picklefile: 
        num_to_word = pickle.load(picklefile)
        
    return target_sentence_list, other_sentence_list, target_vectorized_sentences, other_vectorized_sentences,\
            num_to_word, word_to_num, word_list

In [4]:
target_sentence_list, other_sentence_list, target_vectorized_sentences, other_vectorized_sentences,\
            num_to_word, word_to_num, word_list = import_from_pickle()

In [4]:
def make_subsets(vect):
    train_indices = np.random.choice(len(vect), len(vect)//2, replace=False)
    test_indices = [i for i in range(len(vect)) if i not in train_indices]
    result_test = vect[test_indices]
    result_train = vect[train_indices]
    return result_test, result_train


def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b


def make_test_train(targ_vect_sent, other_vect_sent):
    targ_test, targ_train = make_subsets(targ_vect_sent)
    other_test, other_train = make_subsets(other_vect_sent)
    X_train = np.concatenate((targ_train, other_train))
    X_test = np.concatenate((targ_test, other_test))
    y_train = np.array([1]*len(targ_train)+ [0]*len(other_train))
    y_test = np.array([1]*len(targ_test) + [0]*len(other_test))
    X_train, y_train = shuffle_in_unison(X_train, y_train)
        
    return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, X_test, y_test = make_test_train(target_vectorized_sentences, other_vectorized_sentences)

In [1]:
# def save_as_pickle():
#     with open("X_train.pkl", 'wb') as picklefile: 
#         pickle.dump(X_train, picklefile)
#     with open("X_test.pkl", 'wb') as picklefile: 
#         pickle.dump(X_test, picklefile)
#     with open("y_train.pkl", 'wb') as picklefile: 
#         pickle.dump(y_train, picklefile)
#     with open("y_test.pkl", 'wb') as picklefile: 
#         pickle.dump(y_test, picklefile)

        
# def import_from_pickle():
#     with open("X_train.pkl", 'rb') as picklefile: 
#         X_train = pickle.load(picklefile)
#     with open("X_test.pkl", 'rb') as picklefile: 
#         X_test = pickle.load(picklefile)
#     with open("y_train.pkl", 'rb') as picklefile: 
#         y_train = pickle.load(picklefile)
#     with open("y_test.pkl", 'rb') as picklefile: 
#         y_test = pickle.load(picklefile)
        
#     with open("word_to_num.pkl", 'rb') as picklefile: 
#         word_to_num = pickle.load(picklefile)
#     with open("word_list.pkl", 'rb') as picklefile: 
#         word_list = pickle.load(picklefile)
#     with open("num_to_word.pkl", 'rb') as picklefile: 
#         num_to_word = pickle.load(picklefile)
        
#     return X_train, X_test, y_train, y_test, num_to_word, word_to_num, word_list

In [3]:
#X_train, X_test, y_train, y_test, num_to_word, word_to_num, word_list = import_from_pickle()

## Build and train neural net

In [22]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [7]:
max_review_length = 50
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [8]:
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(len(word_list), embedding_vector_length, input_length=max_review_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=2, batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            5424608   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 5,507,169.0
Trainable params: 5,507,169
Non-trainable params: 0.0
_________________________________________________________________
None


/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 286589 samples, validate on 286589 samples
Epoch 1/2
286589/286589 [==============================] - 1370s - loss: 0.0768 - acc: 0.9721 - val_loss: 0.0517 - val_acc: 0.9793
Epoch 2/2
286589/286589 [==============================] - 1522s - loss: 0.0401 - acc: 0.9840 - val_loss: 0.0475 - val_acc: 0.9808


In [9]:
#model.save('model2epoch.h5')

In [5]:
from keras.models import load_model
model2 = load_model('model2epoch.h5')

Using Theano backend.


### Inspect model for sanity check

In [25]:
def predict_prob_on_sentence(sentence, model):
    word_tokenizer = TreebankWordTokenizer().tokenize
    vec_sentence = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)])])
    padded_sentence = sequence.pad_sequences(vec_sentence, maxlen=50)
    return model.predict_proba(padded_sentence)

def predict_prob_on_paragraph(paragraph, model):
    sentence_list = split_into_sentences(paragraph)
    results = []
    for sentence in sentence_list:
        results.append(predict_prob_on_sentence(sentence, model)[0][0])
    return results

Test it here.

In [29]:
s = 'she was the youngest of the two daughters of a most affectionate, indulgent father'
t= 'the event had every promise of happiness for her friend.'
print(predict_prob_on_sentence(s, model2), predict_prob_on_sentence(t,model2))

1/1 [==============================] - 0s
[[ 0.91311961]] [[ 0.96459585]]


# Doc 2 Vec

In [31]:
def make_doc_2_vec(list_of_sentences, size=100, min_count=5, dm=1, iter=1, workers=4, dbow_words=1):
    '''Text should be a list of sentence strings. Returns doc2vec model and list of tagged sentences'''
    list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(list_of_sentences)]
    docmodel = gensim.models.doc2vec.Doc2Vec(size=size, min_count=min_count, dm=dm, iter=iter, workers=workers, dbow_words=dbow_words)
    docmodel.build_vocab(list_of_tagged_sentences)
    docmodel.train(list_of_tagged_sentences)
    return list_of_tagged_sentences, docmodel

In [64]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel = gensim.models.doc2vec.Doc2Vec(size=100, min_count=5, dm=1, iter=100, workers=16, dbow_words=1)
docmodel.build_vocab(list_of_tagged_sentences)

In [ ]:
len(list_of_tagged_sentences)

573178

In [ ]:
docmodel.train(list_of_tagged_sentences)

In [41]:
v1 = docmodel.infer_vector(list_of_tagged_sentences[28873].words)
v2 = docmodel.infer_vector(list_of_tagged_sentences[27333].words)
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(v1,v2)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.18405372]], dtype=float32)

In [71]:
inferred_vector = docmodel.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(299853, 0.57945716381073), (309920, 0.558485746383667), (294920, 0.5492386817932129), (24318, 0.5309747457504272), (25083, 0.5235756635665894), (552629, 0.5155673027038574), (292002, 0.5140426158905029), (182256, 0.5090835094451904), (307558, 0.5075476765632629), (23924, 0.501731276512146)]
oh, go, go!


then solomon sent benaiah the son of jehoiada, saying, go, fall upon him.


i say run!


go on, go on.


but go on, go on.


i say tomato, you say tomahto.


come at six o'clock and go at six?


walk the walk and talk the talk.


11:38 and he said, go.


fanny, you do not want to go, do you?




In [72]:
inferred_vector = docmodel.infer_vector(['where', 'do', 'you', 'want', 'to', 'go', 'for', 'lunch'])
sims = docmodel.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(325278, 0.584077000617981), (523253, 0.5687791109085083), (299406, 0.5549490451812744), (299853, 0.5469987988471985), (531553, 0.5449169874191284), (2144, 0.5293335318565369), (409651, 0.5240321159362793), (9681, 0.5207614898681641), (156439, 0.5202246904373169), (24318, 0.5169010162353516)]
go and see.


go for lunch, and dinner, and come back the next day.


go, my dear; go and dance again.


oh, go, go!


), solving a dicult puzzle (where does this piece go?


but i dare say you will go for all that.


5%) to induce growth arrest in go, the levels of         gsk3β activity were increased compared to asynchronously         growing cells (fig 4, go).


stay, stay, i will go myself.


why, there's no need even to go outside and collect the newspaper.


go on, go on.




In [73]:
inferred_vector = docmodel.infer_vector(['i', 'love', 'you'])
sims = docmodel.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(283098, 0.6869240403175354), (272756, 0.6668916940689087), (336324, 0.6573529839515686), (280888, 0.6559344530105591), (569856, 0.6473436951637268), (338266, 0.644770622253418), (342868, 0.6348352432250977), (113143, 0.6308819055557251), (557788, 0.6183530688285828), (526547, 0.6179397106170654)]
you, daggoo!


last year, you generously       donated $ .


you arches!


over descartian vortices you hover.


are you a musician?


of you gray rocks!


, 1994; you, et al.


bad tempered and bad for you, says the new                 yorker 's anthony lane.


)  you cursed brat!


you and i have advantages of trade.




In [74]:
docmodel.save('docmodel')

In [80]:
from gensim.models import doc2vec
loadeddocmodel = doc2vec.Doc2Vec.load('docmodel')

In [5]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel3 = gensim.models.doc2vec.Doc2Vec(size=100, min_count=10, iter=30, sample=0.00001, \
                                         workers=16, dm=0, dbow_words=1, seed=0)
docmodel3.build_vocab(list_of_tagged_sentences)

len(list_of_tagged_sentences)

docmodel3.train(list_of_tagged_sentences)

120245845

In [6]:
inferred_vector = docmodel3.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel3.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(179026, 0.819800853729248), (307774, 0.8137213587760925), (54269, 0.8100277185440063), (75487, 0.8056532740592957), (14109, 0.8017879724502563), (72970, 0.8014056086540222), (15435, 0.7987525463104248), (16529, 0.7956112027168274), (299042, 0.7949422597885132), (205454, 0.7938140034675598)]
perhaps a subject for this afternoon?


and they tarried until afternoon, and they did eat both of them.


a few more like this morning's and he'll be kaput.


then, on any given night, you'd run into twice as many of them.


it is a very cold afternoon—but in this carriage we know nothing of the matter.


if you don't want it, you can dispose of it this afternoon.


i think it would be much better if they would come in one afternoon next summer, and take their tea with us—take us in their afternoon walk; which they might do, as our hours are so reasonable, and yet get home without being out in the damp of the evening.


and you must be off this very morning?


who sent us breakfast?


many of our

In [7]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel4 = gensim.models.doc2vec.Doc2Vec(size=100, min_count=10, iter=30, sample=0.0001, \
                                         workers=16, dm=0, dbow_words=1, seed=0)
docmodel4.build_vocab(list_of_tagged_sentences)

len(list_of_tagged_sentences)

docmodel4.train(list_of_tagged_sentences)

inferred_vector = docmodel4.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel4.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(294401, 0.7325917482376099), (34806, 0.7310218214988708), (28906, 0.7276325225830078), (24008, 0.718917965888977), (18470, 0.7148057818412781), (18759, 0.7122348546981812), (19693, 0.7080596685409546), (38518, 0.7055720090866089), (21800, 0.6979830861091614), (287924, 0.6972124576568604)]
may i call again, madam, this afternoon?


she must not walk.


walk!


walk!


—should not they walk?


shall we walk, augusta?


—only that morning.


in the morning .


but any morning will do for this.


is't night?




In [ ]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel5 = gensim.models.doc2vec.Doc2Vec(size=100, min_count=10, iter=30, sample=0.000001, \
                                         workers=16, dm=0, dbow_words=1, seed=0)
docmodel5.build_vocab(list_of_tagged_sentences)

len(list_of_tagged_sentences)

docmodel5.train(list_of_tagged_sentences)

In [10]:
inferred_vector = docmodel5.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel5.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(132764, 0.9279472827911377), (523039, 0.9273142218589783), (231727, 0.927098274230957), (209908, 0.9268016815185547), (238081, 0.9267863035202026), (76230, 0.9253809452056885), (68485, 0.9251948595046997), (235863, 0.9245360493659973), (29689, 0.9236310124397278), (89388, 0.9235737323760986)]
but i wonder if the scientists behind this important philosophical shift have really considered all the consequences of their actions.


i do think my claim is true.


as for something possibly underlying seemingly trivial complaints, there need not necessarily be a repressed anything.


he thinks that these movies are destructive, that they enjoy degradation.


why don't you help me a little, he replied, and not ask me to define the word 'permissive.


getting the police to respond has been useless and seems petty anyway.


but it doesn't explain why the reverse couldn't turn out to be true instead.


that pretty much expresses prudie's thoughts on the matter.


i cannot think how it could happ

In [11]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel6 = gensim.models.doc2vec.Doc2Vec(size=100, min_count=10, iter=30, sample=0.00005, \
                                         workers=16, dm=0, dbow_words=1, seed=0)
docmodel6.build_vocab(list_of_tagged_sentences)
docmodel6.train(list_of_tagged_sentences)

inferred_vector = docmodel6.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel6.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(179026, 0.7787976264953613), (171542, 0.7507870197296143), (29575, 0.7299491167068481), (26806, 0.7297888398170471), (1025, 0.7149198055267334), (63658, 0.7112331986427307), (66809, 0.7084373235702515), (17137, 0.7082470059394836), (4562, 0.7049012184143066), (14980, 0.7034071683883667)]
perhaps a subject for this afternoon?


the weekly e mail delivery should arrive friday afternoon as well.


good morning to you.


i did not arrive here till tuesday evening.


—and are you going this morning?


on the afternoon of oct.


5, 1998)                                           evening.


a walk before breakfast does me good.


— such an evening!


—good morning to you.




In [12]:
combined_sentences = target_sentence_list + other_sentence_list
list_of_tagged_sentences = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(combined_sentences)]
docmodel7 = gensim.models.doc2vec.Doc2Vec(size=100, min_count=10, iter=30, sample=0.000025, \
                                         workers=16, dm=0, dbow_words=1, seed=0)
docmodel7.build_vocab(list_of_tagged_sentences)
docmodel7.train(list_of_tagged_sentences)

inferred_vector = docmodel7.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = docmodel7.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(combined_sentences[docid])
    print()
    print()

[(295613, 0.7797182202339172), (72970, 0.7774734497070312), (552644, 0.7773740291595459), (25865, 0.7767965793609619), (18759, 0.7745453119277954), (318626, 0.7727531790733337), (15516, 0.7718029022216797), (304898, 0.7707496881484985), (33577, 0.7701761722564697), (17137, 0.7686857581138611)]
i think i could walk round him.


if you don't want it, you can dispose of it this afternoon.


we'll get a look at uranus tomorrow!


i am come to walk with you, fanny, said he.


shall we walk, augusta?


2:5 o house of jacob, come ye, and let us walk in the light of the lord.


now i shall really be very happy to walk into the same room with you.


26:12 and i will walk among you, and will be your god, and ye shall be my people.


yes, yes we will have a snug walk together, and i have something to tell you as we go along.


a walk before breakfast does me good.




# OLDER STUFF. CAN IGNORE.

In [ ]:
docmodel.save('docmodel')

In [290]:
v1 = model.infer_vector(list_of_tagged_sentences[28873].words)
v2 = model.infer_vector(list_of_tagged_sentences[27333].words)
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(v1,v2)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.7134791]], dtype=float32)

In [383]:
v1 = model.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
v2 = model.infer_vector(list_of_tagged_sentences[845].words)

cosine_similarity(v1,v2)
model.similarity(['at', 'the'], 'at')

/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 0.88064344,  0.09701519])

In [385]:
inferred_vector = model.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
print(austen_sentences[21514])
sims = model.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

does she go to balls?
[(14931, 0.578771710395813), (17043, 0.5734875798225403), (23826, 0.5693657994270325), (29419, 0.5685966610908508), (34358, 0.5607420206069946), (2144, 0.559876561164856), (845, 0.5560895800590515), (26631, 0.5557641983032227), (27927, 0.5550302863121033), (18996, 0.5496828556060791)]


In [360]:
inferred_vector2 = model.infer_vector(['elizabeth','was','forced','to','go'])
inferred_vector3 = model.infer_vector(['go','and','interfere'])

In [362]:
model.similarity(['elizabeth','was','forced','to','go'],['go','and','interfere'])

ValueError: shapes (5,100) and (3,100) not aligned: 100 (dim 1) != 3 (dim 0)

In [380]:
inferred_vector = model.infer_vector(['i', 'say', 'we', 'go', 'for', 'a', 'walk', 'this', 'afternoon'])
sims = model.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(austen_sentences[docid])
    print()

[(29557, 0.5815350413322449), (28545, 0.5791788697242737), (15817, 0.5709363222122192), (2144, 0.5642344951629639), (34358, 0.563313901424408), (14303, 0.5623300075531006), (21514, 0.5601113438606262), (26855, 0.5595582723617554), (18996, 0.5546441078186035), (845, 0.5527011156082153)]
’ i say, you will come to belle’s wedding, i hope.

i would not be bound to go two miles in it for fifty thousand pounds.

go, and interfere.

but i dare say you will go for all that.

indeed, i do say it.

she tried to stop him; but vainly; he would go on, and say it all.

does she go to balls?

do not say so.

—‘you must go,’ said she.

we must go, said sir john.



In [240]:
austen_sentences = split_into_sentences(austen_text)

In [242]:
tagged = [gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i]) \
          for i, line in enumerate(austen_sentences)]

In [254]:
tagged[0]

TaggedDocument(words=['the', 'family', 'of', 'dashwood', 'had', 'long', 'been', 'settled', 'in', 'sussex'], tags=[0])

In [245]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=50)

In [246]:
model.build_vocab(tagged)

In [247]:
model.train(tagged)

27751403

# Should get rid of " -

In [251]:
inferred_vector = model.infer_vector(['i', 'hope', 'it', 'is', 'a', 'glorious', 'day', 'today'])
sims = model.docvecs.most_similar([inferred_vector], topn=10)
print(sims)

for docid in [docid for docid, sims in sims]:
    print(austen_sentences[docid])
    print()

[(24876, 0.731322705745697), (21756, 0.7059229016304016), (33822, 0.6980264186859131), (29579, 0.6944616436958313), (17172, 0.6897308230400085), (33624, 0.6850894689559937), (28119, 0.6787413954734802), (16466, 0.6751720309257507), (32443, 0.674387514591217), (33181, 0.673367977142334)]
it was a heavy, melancholy day.

the second day’s trial was not so guiltless.

about a fortnight.

and i hope—i hope, miss morland, you will not be sorry to see me.

you do us a great deal of honour to day, i am sure.

we have seen nothing of him since november.

i am sure you would be miserable if you thought so!

miss woodhouse, i hope nothing may happen to prevent the ball.

i wish she had accepted him.

as yet, you have seen nothing of bath.



array([-0.0176868 , -0.00257066,  0.00481869,  0.02462567, -0.02775163,
        0.04307807, -0.00611855, -0.00384286,  0.00371891, -0.02889272,
       -0.00826401,  0.00873966,  0.01128767,  0.0047045 ,  0.00025823,
       -0.05126639, -0.04118882,  0.02682292,  0.04621991, -0.06935307,
        0.01737165, -0.00050234,  0.02227636, -0.00110546,  0.00068362,
       -0.01858515, -0.03066764,  0.0093579 ,  0.04157468,  0.02754266,
       -0.02184875, -0.0104305 , -0.04991533,  0.00858509, -0.00390728,
       -0.04692287,  0.02847849,  0.02653959, -0.00522324,  0.03872556,
       -0.00789808,  0.05382846, -0.02034015, -0.0190276 ,  0.02660172,
       -0.01940051,  0.01538533, -0.01962281, -0.0076128 ,  0.01571602], dtype=float32)

In [66]:
inferred_vector = model.infer_vector(['are', 'you', 'quite', 'sure', 'of', 'it', '?'])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))



In [74]:
sims

[(31557, 0.8228976726531982),
 (32794, 0.8210653066635132),
 (11291, 0.8179731369018555),
 (29305, 0.8168315887451172),
 (11365, 0.8157533407211304),
 (21369, 0.8155891299247742),
 (4559, 0.810936689376831),
 (12018, 0.8092895746231079),
 (14979, 0.8092561960220337),
 (21348, 0.8089516162872314),
 (26539, 0.8086695671081543),
 (11337, 0.8086438179016113),
 (9844, 0.8069813847541809),
 (823, 0.8065879344940186),
 (15236, 0.8055530190467834),
 (11032, 0.8055527210235596),
 (4498, 0.8011966943740845),
 (6704, 0.800519585609436),
 (31856, 0.7996350526809692),
 (28940, 0.7994403839111328),
 (8574, 0.7992561459541321),
 (29788, 0.7990227341651917),
 (19422, 0.7986496090888977),
 (11202, 0.7971107959747314),
 (19608, 0.7967904210090637),
 (29647, 0.7961963415145874),
 (16871, 0.7958846688270569),
 (11239, 0.7943636775016785),
 (2299, 0.7940699458122253),
 (3546, 0.7935033440589905),
 (1276, 0.7934448719024658),
 (25076, 0.793043851852417),
 (11265, 0.7929806709289551),
 (3544, 0.7929611206054

In [ ]:
model.docvecs.most_similar

In [80]:
austen_sentences[21362]

'“There could be no harm in her liking an agreeable man—everybody knew her situation—Mr. Crawford must take care of himself”.'

In [45]:
ranks

[2299,
 1939,
 17904,
 109,
 11505,
 29772,
 7512,
 2578,
 529,
 21362,
 22195,
 1082,
 40,
 4156,
 15087,
 448,
 5591,
 761,
 663,
 21367,
 12760,
 129,
 7443,
 16190,
 8,
 20250,
 5150,
 154,
 963,
 17100,
 24862,
 12683,
 10495,
 32150,
 222,
 7726,
 59,
 20044,
 2895,
 7380,
 1180,
 31563,
 4953,
 1973,
 10073,
 4945,
 13096,
 15149,
 15304,
 10699,
 19947,
 9615,
 2179,
 1465,
 9291,
 4686,
 8514,
 2090,
 540,
 12881,
 20252,
 16320,
 9026,
 19094,
 25581,
 10366,
 20243,
 1149,
 26691,
 37,
 5654,
 8349,
 1460,
 1682,
 1821,
 4074,
 17229,
 10677,
 28376,
 15569,
 12188,
 26699,
 32869,
 6744,
 17058,
 24515,
 713,
 15677,
 1566,
 2991,
 24212,
 265,
 562,
 26365,
 6303,
 435,
 19731,
 2626,
 17708,
 8103,
 1254,
 23130,
 9875,
 148,
 19850,
 29023,
 438,
 13227,
 14732,
 4378,
 9691,
 3072,
 26543,
 9993,
 2274,
 6303,
 1339,
 23987,
 20704,
 10787,
 22,
 6137,
 10556,
 48,
 16511,
 6391,
 239,
 27179,
 26096,
 22531,
 155,
 29554,
 21741,
 1570,
 28589,
 568,
 10950,
 14952,
 1

In [ ]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

### Split  sentences into words in order to build vocab and make mapping between words and integers

In [31]:
from nltk.tokenize.treebank import TreebankWordTokenizer

def vectorize_text(target_text, other_text):
    target_sentence_list = split_into_sentences(target_text)
    other_sentence_list = split_into_sentences(other_text)
    print('num of target, other sentences', len(target_sentence_list), len(other_sentence_list))
    
    word_tokenizer = TreebankWordTokenizer().tokenize
    word_set = set()
    for sentence in target_sentence_list+other_sentence_list:
        word_set = word_set.union(set(word_tokenizer(sentence)))
    print('number of words in vocab', len(word_set))
    word_list = list(word_set)
    num_to_word = {j:word for j, word in enumerate(word_list)}
    word_to_num = {word:j for j, word in enumerate(word_list)}
    
    target_vectorized_sentences = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)]) \
                                            for sentence in target_sentence_list])
    
    other_vectorized_sentences = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)]) \
                                            for sentence in other_sentence_list])
    
    return target_vectorized_sentences, other_vectorized_sentences, num_to_word, word_to_num, word_list

In [ ]:
target_vectorized_sentences, other_vectorized_sentences, num_to_word, word_to_num, word_list \
    = vectorize_text(austen_text, other_text)

num of target, other sentences 35012 547017


In [ ]:
def make_subsets(vect):
    train_indices = np.random.choice(len(vect), len(vect)//2, replace=False)
    test_indices = [i for i in range(len(vect)) if i not in train_indices]
    result_test = vect[test_indices]
    result_train = vect[train_indices]
    return result_test, result_train


def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b


def make_test_train(targ_vect_sent, other_vect_sent):
    targ_test, targ_train = make_subsets(targ_vect_sent)
    other_test, other_train = make_subsets(other_vect_sent)
    X_train = np.concatenate((targ_train, other_train))
    X_test = np.concatenate((targ_test, other_test))
    y_train = np.array([1]*len(targ_train)+ [0]*len(other_train))
    y_test = np.array([1]*len(targ_test) + [0]*len(other_test))
    X_train, y_train = shuffle_in_unison(X_train, y_train)
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = make_test_train(target_vectorized_sentences, other_vectorized_sentences)

## Time for  RNN

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [47]:
!ls

austenbooks	imdb.ipynb     Untitled1.ipynb	X_test.pkl
Austen.ipynb	moby_dick.txt  Untitled.ipynb	X_train.pkl
get_text.ipynb	OANC-GrAF      untitled.txt	y_train.pkl


In [44]:
import pickle

with open('X_train.pkl', 'wb') as picklefile:
    pickle.dump(X_train, picklefile)

In [49]:
with open('y_test.pkl', 'wb') as picklefile:
    pickle.dump(y_test, picklefile)

In [50]:
!ls

austenbooks	imdb.ipynb	 Untitled.ipynb  X_train.pkl
Austen.ipynb	moby_dick.txt	 untitled.txt	 y_test.pkl
get_text.ipynb	Untitled1.ipynb  X_test.pkl	 y_train.pkl


In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(len(word_list), embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
l=[]
l.append(model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=5, batch_size=128))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           8764672   
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 8,847,233.0
Trainable params: 8,847,233
Non-trainable params: 0.0
_________________________________________________________________
None


/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 291014 samples, validate on 291015 samples
Epoch 1/5
 70400/291014 [======>.......................] - ETA: 8083s - loss: 0.1377 - acc: 0.9579

In [ ]:
l

In [24]:
hist.history

{'acc': [0.77744712509299607, 0.94590285896482096],
 'loss': [0.45427668586853193, 0.14299210789239769],
 'val_acc': [0.86408076514346444, 0.93421891604675877],
 'val_loss': [0.27790165316238286, 0.15308598510037186]}

In [73]:
def predict_prob_on_sentence(sentence, model):
    word_tokenizer = TreebankWordTokenizer().tokenize
    vec_sentence = np.array([np.array([word_to_num[word] for \
                                               word in word_tokenizer(sentence)])])
    padded_sentence = sequence.pad_sequences(vec_sentence, maxlen=max_review_length)
    return model.predict_proba(padded_sentence)

def predict_prob_on_paragraph(paragraph, model):
    sentence_list = split_into_sentences(paragraph)
    results = []
    for sentence in sentence_list:
        results.append(predict_prob_on_sentence(sentence, model)[0][0])
    return results

In [ ]:
paragraph = """“I do not understand what you mean by ‘success,’” said Mr. Knightley. “Success supposes endeavour. Your time has been properly and delicately spent, if you have been endeavouring for the last four years to bring about this marriage. A worthy employment for a young lady’s mind! But if, which I rather imagine, your making the match, as you call it, means only your planning it, your saying to yourself one idle day, ‘I think it would be a very good thing for Miss Taylor if Mr. Weston were to marry her,’ and saying it again to yourself every now and then afterwards, why do you talk of success? Where is your merit? What are you proud of? You made a lucky guess; and that is all that can be said.”"""

In [74]:
predict_prob_on_paragraph(paragraph, model)

1/1 [==============================] - 0s


[0.98919564,
 0.50044167,
 0.84341258,
 0.81804514,
 0.96195269,
 0.37761614,
 0.47974524,
 0.56973124]